In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

zsh:1: command not found: wget


In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read() 

In [5]:
print(len(text))

1115394


In [6]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [7]:
chars = sorted(list(set(text)))

In [8]:
vocab_size = len(chars)

In [9]:
print(vocab_size)

65


In [10]:
for i in range(65): print(chars[i], end='')


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

In [11]:
str_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_str = { i:ch for i,ch in enumerate(chars) }

In [12]:
encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_str[i] for i in l])

In [13]:
print(encode("llm mystery remains"))

[50, 50, 51, 1, 51, 63, 57, 58, 43, 56, 63, 1, 56, 43, 51, 39, 47, 52, 57]


In [14]:
print(decode(encode("llm mystery remains")))

llm mystery remains


In [25]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [26]:
n = int(0.9*len(data)) 
train_data = data[:n]
val_data = data[n:]

In [27]:
block_size = 8

In [28]:
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [20]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [21]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs :")
print(xb.shape)
print(xb)
print("targets :")
print(yb.shape)
print(yb)

print("------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target  = yb[b,t]
        print(f"{context.tolist()},{target}")

inputs :
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets :
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------
[24],43
[24, 43],58
[24, 43, 58],5
[24, 43, 58, 5],57
[24, 43, 58, 5, 57],1
[24, 43, 58, 5, 57, 1],46
[24, 43, 58, 5, 57, 1, 46],43
[24, 43, 58, 5, 57, 1, 46, 43],39
[44],53
[44, 53],56
[44, 53, 56],1
[44, 53, 56, 1],58
[44, 53, 56, 1, 58],46
[44, 53, 56, 1, 58, 46],39
[44, 53, 56, 1, 58, 46, 39],58
[44, 53, 56, 1, 58, 46, 39, 58],1
[52],58
[52, 58],1
[52, 58, 1],58
[52, 58, 1, 58],46
[52, 58, 1, 58, 46],39
[52, 58, 1, 58, 46, 39],58
[52, 58, 1, 58, 46, 39, 58],1
[52, 58, 1, 58, 46, 39, 58, 1],46
[25],17
[25, 17],27
[25, 17, 27],10
[25, 17, 27, 10],0
[25, 17, 27, 10, 0],21
[25, 17, 27, 10, 0

In [22]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [29]:
# bi-gram language model

In [30]:
import torch.nn as nn
from torch.nn import functional as F

In [31]:
torch.manual_seed(1337)

In [45]:
class bigramlangmodel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        #token reads the next token's logit through look up table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens): #idx--{B,T} format
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:] # used to focus on the last step
            probs = F.softmax(logits,dim=-1) #softmax for probabs
            idx_next = torch.multinomial(probs, num_samples =1)
            #now to append the generated sample
            idx = torch.cat((idx, idx_next), dim = 1)
            
        return idx
    
m = bigramlangmodel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(logits, loss)


print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens = 1000)[0].tolist()))

torch.Size([32, 65])
tensor([[ 6.0637e-01, -1.8221e+00, -1.1690e+00,  ..., -1.1942e+00,
         -6.2601e-01,  3.1012e-01],
        [ 7.1723e-01,  9.7372e-02, -6.3213e-02,  ..., -1.7488e-01,
         -8.4489e-01, -1.1427e+00],
        [ 3.7405e-01, -5.2491e-01, -9.1027e-01,  ..., -8.6256e-01,
         -1.2455e+00,  6.6196e-04],
        ...,
        [ 3.3056e-01,  7.7441e-01,  2.0565e+00,  ...,  1.3960e+00,
          1.5854e+00, -1.0350e+00],
        [ 1.0631e-01,  1.7815e-01, -2.5728e-01,  ...,  7.0672e-01,
         -3.3647e-01, -4.4001e-01],
        [ 1.5161e+00,  1.6629e+00,  1.3459e+00,  ..., -1.2560e+00,
         -8.5629e-02,  5.8109e-01]], grad_fn=<ViewBackward0>) tensor(4.7854, grad_fn=<NllLossBackward0>)

hml CHZM!X&rrNtT3 M-A'ky$cIMU&ro--A
onOW3w-EFVqoEbCBalnof3qqOVCN&QRk:'IYeZAvcsJ.OlXEjUme!ftouvYb&QB;wDY;,nAHEE?UI?CRcD:VMZjC$Uvu.yICRpHP'vJgsvFF??oJ3PHPmEfWH.-Nl O
Rn.PDLzXAS3,nZMDlJ:zpG!rHM&FVuG3y---AvIdsqrrbKzOBln
MQcgh.MNAaDxMqQ3Ng??qOzB,BHxV-AvYkuuKRc!S,PWLsgaSihr;u.OkyxeOY

In [ ]:
# training the bigram model

In [46]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [47]:
batch_size = 32
for steps in range(100):
    xb,yb = get_batch("train")
    
    # loss evaluate
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    print(loss.item())
    

4.5635175704956055
4.607728481292725
4.568040370941162
4.558488368988037
4.523165702819824
4.54990816116333
4.603701114654541
4.608258247375488
4.4874114990234375
4.600671291351318
4.671968936920166
4.469656467437744
4.541782379150391
4.530789375305176
4.565713405609131
4.489282131195068
4.615777015686035
4.533841133117676
4.549324035644531
4.497089385986328
4.499934673309326
4.548351287841797
4.496674060821533
4.513086318969727
4.441937446594238
4.470240116119385
4.468621730804443
4.540816307067871
4.489208221435547
4.337831020355225
4.577211856842041
4.41691780090332
4.500166416168213
4.557855606079102
4.463111877441406
4.4933881759643555
4.560523986816406
4.5298171043396
4.463313579559326
4.589207172393799
4.543258190155029
4.503807544708252
4.516111850738525
4.5566792488098145
4.5433478355407715
4.560929298400879
4.614007949829102
4.481178283691406
4.488757133483887
4.4382195472717285
4.5524678230285645
4.443222522735596
4.538846492767334
4.48873233795166
4.574199199676514
4.622672

In [48]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


PQi
NOLzQBLIyybrUc,INVBhKFvBZ,-U&lng&ma&mJ NYcfFKjFCqa C$To-uO
M&gaqdN&NQwYoJO
Mm W;CZgH;epcQqkJeBzCU!kcWroZ!xOBqFAkyJSU:pH.'!
qAxGBhVaDAWn?: mZxk:;OxT RcqZMqa&Z-EWv!gfNvC--hYxabJBJnY--AorrZxBZ-AWrroxMao-AMz!MaDexkyHnglacIZ:MAH;p-iOIJKMMuYxjLIUHoxB'K W:zq Lrr.Lii$Ixbd$?P-VtoYxb&GTZe$H?IacIxnoS;xaiM!!&QrbKyh&j.CR3&NlXstbQ'OKftUIl&afqFU'X NGMUwIcqihTrIlnkm&CRnRVaYSwIBvLIaer.:zVKXjUneHoJ:twST iMMOSZvQgaWPj-UDGKIHt
Dts Pfu.JLIUr.usGvwyKy:!jU
jM&aS$UmyNljL?ubVKdiFaO!DenmXI'UItF:B-T-tSS'zIxVKf3xLuamm'


In [49]:
batch_size = 32
for steps in range(1000): 
    xb,yb = get_batch("train")
    
    # loss evaluate
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())


#1000

3.614607810974121


In [50]:
batch_size = 32
for steps in range(1000):
    xb,yb = get_batch("train")
    
    # loss evaluate
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())


#1000

3.1206002235412598


In [52]:
%%time
batch_size = 32
for steps in range(10000):
    xb,yb = get_batch("train")
    
    # loss evaluate
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print("loss = ",loss.item())


#10,000

loss =  2.479030132293701
CPU times: user 10.1 s, sys: 3.92 s, total: 14.1 s
Wall time: 10.2 s


In [53]:
# now testing the model 

In [54]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))



Ande,
Therak's y, f be t se siakikice ay e pu y omufid vesll ampr;
MONomy
Gutin, ingt besind co,
H: fiovers urataveridor, w, Le
Prsethat qurcoutheat, f at mid?
RD wan'sthanthid had:
ve u?


hobeesthifo,

TENI wie ou mesis, h, wng ss g, uiverr h be, lourt
O:
I t ine, t h?
Thyond w be,
Toullexa hano sbeiggure, maitoto?

An t con:

Totint owivear, res, wadinkimove PENatik Samaketh CETICa'sl wiothe ll ove pre dsik tth:
TI m,
PR:
Fand oololote thit w bede, mathe pus laimse g, ind fase r chitoris P:



In [56]:
#it shows less gibberrish now xd

In [57]:
# SELF ATTENTION HEAD

In [58]:
#1 Averaging past context -- very unefficient

In [60]:
torch.manual_seed(1337)
B, T, C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [61]:
#for loop
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # till the t its taking means including that position also
        xbow[b,t] = torch.mean(xprev, 0)

In [62]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [63]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [64]:
# can see that every location is averaged in xbow[] of the prev position and the position itself included

In [65]:
# version 2 using matrix multiply 

In [67]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a,1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a@b
print(a)
print("---")
print(b)
print("---")
print(c)
print("---")

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
---


In [68]:
wei = torch.tril(torch.ones(T,T))
wei = wei/torch.sum(wei,1,keepdim=True)
wei


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [69]:
xbow2 = wei @ x
#(T,T)matmul(B,T,C) where (T,T) will be changed into(torch will change it to) (B,T,T)@(B,T,C)--->((B,T,C))

In [70]:
torch.allclose(xbow,xbow2)

True

In [71]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [73]:
xbow2[0] #both are same but this efficient

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [74]:
# version 3 using softmax

In [78]:
trill = torch.tril(torch.ones(T,T))
trill

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [80]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(trill == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [81]:
# exactly the same matrix as ver1 && ver2

In [83]:
xbow3 = wei @ x
torch.allclose(xbow3,xbow2)

True